motivation: 
- __YTHDC1__ is known to physically interact with __SETDB1__. They both show up as sensetizing hits in Alex's DAC CRISPRi screen. 

aim:
- Thus, I'm doing non-discovery mode motif analysis of 

### [Flexible Binding of m6A Reader Protein YTHDC1 to Its Preferred RNA Motif](https://pubs.acs.org/doi/10.1021/acs.jctc.9b00987)

    YTHDC1 -> 5′-G–2G–1(m6A)C+1U+2-3′

### [RNA methylation preserves ES cell identity by chromatin silencing of retrotransposons](https://www.nature.com/articles/s41392-021-00683-4)
> Coimmunoprecipitation revealed physical interactions between YTHDC1 and SETDB1, suggesting that m6A-binding YTHDC1 recruits SETDB1 to methylate H3K9 at specific retrotransposons ([Fig. 1](https://www.nature.com/articles/s41392-021-00683-4/figures/1)).

> Working model for the mechanism underlying m6A and YTHDC1-mediated maintenance of embryonic stem cell identity through chromatin silencing. Methylation of retrotransposon RNA recruits m6A reader YTHDC1 and H3K9me3 methyltransferase SETDB1 to promote retrotransposon chromatin silencing, which represses two-cell-like state transition, leading to embryonic stem cell identity maintenance


<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41392-021-00683-4/MediaObjects/41392_2021_683_Fig1_HTML.png" width=500 />

In [4]:
from IPython.display import IFrame

### imRIP pipeline

In [20]:
!cat ~/Workflows/imRIP/motifs.txt

[AG]GAC
[AGT][AG]AC[ACT]


In [21]:
!cat ~/Workflows/imRIP/scr/exomepeak-motif.sh

MAIN=$1
MOTIF=/rumi/shams/abe/GitHub/imRIP/motifs.txt

cd ${MAIN}
for sam in *; do
	cd $sam
	echo "__________________________________________________________________________________________"
	echo $sam
	echo "step 1: extract mRNA sequences"
	cat peak.bed | sort -k1,1 -k2,2n peak.bed | cgat bed2bed --method=merge --merge-by-name |  awk '! /#/' | bedtools getfasta -name -s -fi /rumi/shams/genomes/hg38/hg38.fa -bed - -split -fo peak.fa
	echo "--- DONE! ---"

	echo "step 2: prepare inputs for FIRE"
	# perl $TEISERDIR/prep_seqs_for_teiser_run.pl peak.fa peaks
	/rumi/shams/abe/anaconda3/envs/cgat/bin/python $TEISERDIR/prep_fasta_for_fire_run.py peak.fa
	echo "--- DONE! ---"

	echo "step 3: run FIRE for known m6A motifs (non-discovery mode)"
	perl $FIREDIR/fire.pl --expfile=peak_fire.txt --exptype=discrete --fastafile_rna=peak_fire.fa \
	--nodups=1 --dodna=0 --dodnarna=0 --species=human --doskipdiscovery=1 \
	--motiffile_rna=$MOTIF --oribiasonly=0 > non-discovery_FIRE.log
	rm -rv non-discover

___
I only need "step3" here, I already made the `.fasta` file for radar peaks – i.e. decitabine induced differentially m6A methylated peaks :)

In [81]:
cat radar/YTH_motifs.txt

GGACT
[AG][AG]AC[ACT]

In [83]:
%%bash
export FIREDIR=/data_gilbert/home/aarab/Workflows/FIRE-1.1
PEAKS_exp=result.all_fire.txt
PEAKS_fa=result.all_fire.fa

MOTIF="YTH_motifs.txt"

echo "step 3: run FIRE for known m6A motifs (non-discovery mode)"

cd radar
perl $FIREDIR/fire.pl \
    --expfile=$PEAKS_exp --exptype=discrete \
    --fastafile_rna=$PEAKS_fa \
    --nodups=1 --dodna=0 --dodnarna=0 --species=human --doskipdiscovery=1 \
    --motiffile_rna=$MOTIF \
    --oribiasonly=0 &> /dev/null
#> non-discovery_FIRE.log

rm -rv YTHDC1motifs_FIRE
mv -v ${PEAKS_exp}_FIRE/ YTHDC1motifs_FIRE 

cd ../
echo "--- DONE! ---"


step 3: run FIRE for known m6A motifs (non-discovery mode)
removed ‘YTHDC1motifs_FIRE/cmdline.txt’
removed directory: ‘YTHDC1motifs_FIRE/DNA’
removed ‘YTHDC1motifs_FIRE/RNA/result.all_fire.txt’
removed directory: ‘YTHDC1motifs_FIRE/RNA/result.all_fire.txt.mimatrix_OUT’
removed ‘YTHDC1motifs_FIRE/RNA/result.all_fire.txt.script’
removed ‘YTHDC1motifs_FIRE/RNA/result.all_fire.txt.optim’
removed directory: ‘YTHDC1motifs_FIRE/RNA’
removed directory: ‘YTHDC1motifs_FIRE/DNA_RNA’
removed directory: ‘YTHDC1motifs_FIRE’
‘result.all_fire.txt_FIRE/’ -> ‘YTHDC1motifs_FIRE’
--- DONE! ---


In [87]:
IFrame("radar/YTHDC1motifs_FIRE/RNA/result.all_fire.txt.summary.pdf", width=1200, height=500)

In [ ]:
%%bash
export FIREDIR=/data_gilbert/home/aarab/Workflows/FIRE-1.1
PEAKS_exp=result.all_fire.txt
PEAKS_fa=result.all_fire.fa

echo "step 4: run FIRE discovery mode"

cd radar
perl $FIREDIR/fire.pl \
    --expfile=$PEAKS_exp --exptype=discrete --fastafile_rna=$PEAKS_fa \
    --nodups=1 --dodna=0 --dodnarna=0 --species=human --oribiasonly=0

rm -rv discovery_FIRE 
mv -v ${PEAKS_exp}_FIRE/ discovery_FIRE

cd ../
echo "--- DONE! ---"

In [ ]:
IFrame("radar/result.all_fire.txt_FIRE/RNA/result.all_fire.txt", width=1200, height=500)

# 

In [12]:
!echo "TACAGAGAA" > radar/MAT2A_hairpin_motif.txt

In [14]:
%%bash
export FIREDIR=/data_gilbert/home/aarab/Workflows/FIRE-1.1
PEAKS_exp=result.all_fire.txt
PEAKS_fa=result.all_fire.fa

MOTIF="MAT2A_hairpin_motif.txt"

echo "step 3: run FIRE for known m6A motifs (non-discovery mode)"

cd radar
perl $FIREDIR/fire.pl \
    --expfile=$PEAKS_exp --exptype=discrete \
    --fastafile_rna=$PEAKS_fa \
    --nodups=1 --dodna=0 --dodnarna=0 --species=human --doskipdiscovery=1 \
    --motiffile_rna=$MOTIF \
    --oribiasonly=0 &> /dev/null
#> non-discovery_FIRE.log

rm -r MAT2A_hairpin_motif_FIRE
mv -v ${PEAKS_exp}_FIRE/ MAT2A_hairpin_motif_FIRE 

cd ../
echo "--- DONE! ---"


step 3: run FIRE for known m6A motifs (non-discovery mode)
‘result.all_fire.txt_FIRE/’ -> ‘MAT2A_hairpin_motif_FIRE’
--- DONE! ---


In [16]:
IFrame(
    "radar/MAT2A_hairpin_motif_FIRE/RNA/result.all_fire.txt.summary.pdf", 
    width=1200, height=500
)

In [48]:
!date

Mon Jan  3 18:48:39 PST 2022
